
# Matrix Transpose

In [1]:
import d2ltvm
import tvm
from tvm import te
import numpy as np

successd...



In [38]:
dtype = 'float32'
n = te.var(name='n')
m = te.var(name='m')
A = te.placeholder(shape=(n,m), name='a', dtype=dtype)
B = te.compute((m,n), lambda i, j: A[j][i], name='b')
s = te.create_schedule(B.op)

In [40]:

print(tvm.lower(s, [A, B]).show())

None


In [41]:
def test_matrix_transpose(mod, shape, dtype):
    a = np.arange(np.prod(shape)).reshape(shape).astype(dtype)
    b = np.empty(shape[::-1]).astype(dtype)
    a, b = tvm.nd.array(a), tvm.nd.array(b)
    mod(a, b)
    return a, b
    
mod = tvm.build(s, [A, B])
shape = (3,4)
a, b = test_matrix_transpose(mod, shape, dtype)
print(a)
print(b)

[[ 0.  1.  2.  3.]
 [ 4.  5.  6.  7.]
 [ 8.  9. 10. 11.]]
[[ 0.  4.  8.]
 [ 1.  5.  9.]
 [ 2.  6. 10.]
 [ 3.  7. 11.]]


# Reshaping

In [46]:
B = te.compute((m * n,), lambda i: A[i // m, i % m], name='b')
s = te.create_schedule(B.op)
print(tvm.lower(s, [A,B], simple_mode=True).show())

None


In [78]:
# (n,m) ---> (p,q)
p = te.var(name='p')
q = te.var(name='q')
B = te.compute((p,q), lambda i, j: A[((i * q) + j) // m, ((i * q) + j) % m], name='b')
s = te.create_schedule(B.op)
print(tvm.lower(s, [A, B], simple_mode=True))


@main = primfn(a_1: handle, b_1: handle) -> ()
  attr = {"from_legacy_te_schedule": True, "global_symbol": "main", "tir.noalias": True}
  buffers = {a: Buffer(a_2: Pointer(float32), float32, [(stride: int32*n: int32)], [], type="auto"),
             b: Buffer(b_2: Pointer(float32), float32, [(stride_1: int32*p: int32)], [], type="auto")}
  buffer_map = {a_1: a, b_1: b}
  preflattened_buffer_map = {a_1: a_3: Buffer(a_2, float32, [n, m: int32], [stride, stride_2: int32], type="auto"), b_1: b_3: Buffer(b_2, float32, [p, q: int32], [stride_1, stride_3: int32], type="auto")} {
  for (i: int32, 0, p) {
    for (j: int32, 0, q) {
      b[((i*stride_1) + (j*stride_3))] = a[((floordiv(((i*q) + j), m)*stride) + (floormod(((i*q) + j), m)*stride_2))]
    }
  }
}




In [83]:
# te.var只在定义op算子是出现，编译好的mod输入的是固定的数值，由build参数决定
mod = tvm.build(s, [A, B, p, q])
bshape = (1, 12)
b = np.empty(bshape).astype(dtype)
b = tvm.nd.array(b)
mod(a, b, *bshape)
print(a)
print(b)

[[ 0.  1.  2.  3.]
 [ 4.  5.  6.  7.]
 [ 8.  9. 10. 11.]]
[[ 0.  1.  2.  3.  4.  5.  6.  7.  8.  9. 10. 11.]]


# Slicing

In [105]:
def ceil(a, b):
    return -1 * (-a // b)

In [113]:
# A[bi::si,bj::sj]
bi, bj, si, sj = [te.var(name) for name in ['bi', 'bj', 'si', 'sj']]
B = te.compute((ceil((n - bi), si), ceil((m - bj), sj)), lambda i, j: A[i * si + bi, j * sj + bj], name='b')
s = te.create_schedule(B.op)
mod = tvm.build(s, [A, B, bi, si, bj, sj])

b = tvm.nd.array(np.empty((2, 2), dtype='float32'))
print(a.asnumpy()[1::1,1::2])
print(b)

[[ 5.  7.]
 [ 9. 11.]]
[[ 5.  7.]
 [ 9. 11.]]


# Summary
1.Both shape dimensions and indices can be expressions with variables

2.If a variable doesn’t only appear in the shape tuple, we need to pass it as an argument when compiling